# Solar Data Tutorial Worksheet 
Hello Solar REU interns at the CfA. This is a quick data tutorial, which hopefully will help with your summer reasearch.

## Helioviewer Example
Helioviewer is a useful tool for exploration. Using JHelioviewer in particular may help you find interesting events. Here I will walk you through a quick example with JHelioviewer. 

You can launch JHelioviewer by clicking the JHelioviewer icon (below) in either Applications or your Dock.
<img src="JHV_icon_small.png" />


# Data Download 
There are two primary ways to download data for Solar mission archives: websites and programming scripts.

## WEBSITES
### Virtual Solar Observatory ([VSO](https://sdac.virtualsolar.org/cgi/search))
 1. Allows you to select certain parameters when looking for data
 <img src="vso_pic/vso_search_form.png" />
 2. Then click generate form
 3. Select time range of interest (Currently in upper right)
 <img src="vso_pic/vso_time_range.png" />
 3. Select observatories to get data from (e.g. IRIS, AIA, XRT)
 <img src="vso_pic/vso_select_instr.png" />
 4. Select interested wavelenths (at bottom)
 <img src="vso_pic/vso_select_wavelengths.png" />
 5. When you are done click search (back in the time box)
 <img src="vso_pic/vso_time_range.png" />
 6. Select observations
   1. One at a time 
   <img src="vso_pic/vso_select_observations.png" />
   2. All (on left hand side)
   <img src="vso_pic/vso_select_all_box.png" />
 7. Request data from VSO (bottom left)
   <img src="vso_pic/vso_request_data.png" />
 8. Download Data
   1. Click on data to download **OR**
   2. Click Export Links at bottom left and download via the following command where YYYY... is specific to your file
      1. tcsh cat vso_export_YYYYMMDD_HHMMSS.csv | xargs wget




### Joint Science Operations Center ([JSOC](http://jsoc.stanford.edu/ajax/lookdata.html))
  1. Focused more on SDO observations but also has IRIS observations
  2. Add text to the series filter name (e.g. aia, hmi, or iris) then click fetch seriesname list
  <img src="jsoc_pic/jsoc_search_series.png" />
  3. Select a series, by clicking, from the list (I chose aia.lev1_euv_12s, which is the LEVEL 1 SDO/AIA EUV observations at a 12s cadence)
  <img src='jsoc_pic/jsoc_select_series.png' />
  4. Create a query for JSOC
    1. Here I created a query of LEVEL 1 SDO/AIA EUV observations (aia.lev1_euv_12s) for 5 minutes (/5m) starting on September 8th, 2017 at 10:59:59 UTC (2017-09-08T10:59:59Z) at a 30 second cadence (@30s) using only the 171, 193, and 211 wavelengths ([171, 193, 211]). Then I clicked GetRecordCount to get the number of observations I am requesting (27).
  <img src='jsoc_pic/jsoc_record_select_fix.png' />
  5. If the request number is similar to what you expect (e.g. 2 observations per minute x 5 minutes x 3 wavelenths per observation = 30) then click Export Data in the top tool bar.
  <img src='jsoc_pic/jsoc_click_export_data.png' />
  6. Then click export again to be pedantic (don't worry you are not done clicking export yet)
  <img src='jsoc_pic/jsoc_click_export_fix.png' />
  7. Check the record count one more time (make sure in the example it is still 27)
  8. Choose the method for delivery. I usually choose url-tar or ftp-tar because it packages everything nicely.
  9. Fill in Notify with your email address (on your first attempt it will send you an email for verification)
  10. Click Check Params for Export.
  11. If everything is good click Submit Export Request (will be green after clicking Check Params for Export succeeds)
  <img src='jsoc_pic/jsoc_export_request_generation_fix.png' />
  12. Now check the JSOC Data Export Status and Retrieval at the bottom of the page and store the Request ID somewhere. I will save you in case your browser crashes or you accidentally close your browser. 
  13. Finally, you should get an email with the link, which you can click to download.
  14. Sometimes you do not get an email, in which case keep the tap open and sparingly click Submit Status Request until the data is ready. Then click the link in Tar File Location.
  <img src='jsoc_pic/jsoc_export_final.png' />


## Progaming Scripts

### IDL (SSWIDL)
;Starting time   
*t1 = '2017/09/08T11:00:00'*   
;Ending time   
*t2 = '2017/09/08T16:00:00'*   
;Cadence to get data in seconds (30 minutes)   
_cad = 30.*60._    

;Search the VSO (previously used as a website) for SDO/HMI (inst='hmi') observations from JSOC (provider='jsoc') for line of sight magentic fields (physobs='los_magnetic_field') at a 30 minute cadence (sample=cad)   
*s_f = vso_search(t1,t2,inst='hmi',provider='jsoc',physobs='los_magnetic_field',sample=cad)* 

;Or do the same thing for SDO/AIA observations   
;Wavelengths of interest   
wave = ['193','304','335']   
;Search the VSO (previously used as a website) for SDO/AIA (inst='aia') observations from JSOC (provider='jsoc') for intensity observations (physobs='intensity') for wavelenths of 193, 304, and 335 Angstroms (wave=wave) at a 30 minute cadence (sample=cad)  
*s_f = vso_search(t1,t2,inst='aia',provider='jsoc',physobs='intensity',sample=cad,wave=wave)*


;The s_f variables contain the list of files to download from the VSO. To actually download you need two more lines.  
;Set the output directory   
*aia_arch = 'aia_arch/'*    
;Download files to the output directory   
*s_r = vso_get(s_f,out_dir=aia_arch,/FORCE)*

;You can get more information by typing the following command   
*xdoc,'vso_search'*

### python ([Sunpy](http://docs.sunpy.org/en/v0.9.0/generated/gallery/skip_searching_vso.html))

In [5]:
from sunpy.net import Fido, attrs
from datetime import datetime, timedelta
import astropy.units as u
import os


#Set a time of interest 2017/09/08 12:00:00 UTC
dttime = datetime(2017,9,8,12,0,0)

#output format for time string
sform = '%Y/%m/%d %H:%M:%S'

#Get time range around a given time
dt = timedelta(seconds=60)

#Get two minutes around dttime and covert to a string
start = datetime.strftime(dttime-dt,sform)
end = datetime.strftime(dttime+dt,sform)
    
#set time span
time = attrs.Time(start,end)
#grabs both stereo a and b
ins = attrs.Instrument('aia')
#grab 171 wavelength
wave171 = attrs.Wavelength(171*u.AA)
#grab 193 wavelength
wave193 = attrs.Wavelength(193*u.AA)



In [8]:
#Get list of files from VSO
result = Fido.search(time,ins, wave171 | wave193)
#download the files to the local directory
out_path = os.getcwd()+"/"
#Download the files
dl_fil = Fido.fetch(result,path=out_path,wait=True)

[                                        ========================================
[                                        ========================================
